In [73]:
import pandas as pd
import numpy as np
import bs4 
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split, StratifiedKFold
from sklearn import metrics 
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
!pip install imblearn

Выгружаем данные предварительно собранные с сайта irecommend

In [74]:
data_reviews = pd.read_csv('output_irecommend.csv')

In [75]:
data_reviews.head()

label                                               text
0  рекомендует  <p>Добрый день! Сегодня хотел бы поделиться от...
1  рекомендует  <p>Стильный, шустрый, подойдёт как для игр так...
2  рекомендует  <p>Был такой телефон в пользовании доволен кач...
3  рекомендует  <p>Пользуюсь смартфоном уже год. Ни разу не по...
4  рекомендует  <p>Расскажу про свою покупку с Ибея! Ей стал с...

In [76]:
data_reviews.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6781 entries, 0 to 6780
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   6781 non-null   object
 1   text    6111 non-null   object
dtypes: object(2)
memory usage: 106.1+ KB


Обрабатка данных

In [77]:
data_reviews.dropna(inplace=True)

In [78]:
data_reviews['label'].unique()

array(['рекомендует', 'не рекомендует'], dtype=object)

In [79]:
data_reviews['label'].replace(['рекомендует', 'не рекомендует'], [1,0], inplace=True)

In [80]:
data_reviews['text'] = data_reviews['text'].apply(lambda x: bs4.BeautifulSoup(x).get_text())

In [81]:
data_reviews

label                                               text
0         1  Добрый день! Сегодня хотел бы поделиться отзыв...
1         1  Стильный, шустрый, подойдёт как для игр так и ...
2         1  Был такой телефон в пользовании доволен качест...
3         1  Пользуюсь смартфоном уже год. Ни разу не подво...
4         1  Расскажу про свою покупку с Ибея! Ей стал смар...
...     ...                                                ...
6776      1  Расскажу о своем недавнем приобретении.,Я реши...
6777      1   И все бы не чего, да вот телефон мой (а на то...
6778      1  С этого телефона началось моё знакомство с фир...
6779      1  Доброго времени суток!,Больше года назад мой с...
6780      1  Уже почти год пользуюсь этим смартфоном и гото...

[6111 rows x 2 columns]

In [82]:
data_reviews.loc[3, 'text']

'Пользуюсь смартфоном уже год. Ни разу не подводил. Работает быстро, не зависает, обновляется регулярно. Из достоинств можно подчеркнуть отличную камеру, яркий дисплей, хорошая связь. Регулярно устанавливаются хорошие программы после обновления, которые без рекламы. Например, Quick. Поддерживает быструю зарядку (Заряжается до 80-90% за 30-40 минут). Наушники в комплекте отличные. Держит заряд 1 день при условии обычного пользования.'

Создание модели

In [83]:
data_reviews['label'].value_counts()

1    5710
0     401
Name: label, dtype: int64

Создание матрицы признаков

In [84]:
def pipeline_preprocessing_features(vectorizer, transformer):
    return Pipeline([('vectorizer', vectorizer),
                    ('transformer', transformer)])


In [113]:
from nltk.corpus import stopwords
data_transformer = pipeline_preprocessing_features(CountVectorizer(ngram_range=(1,3),
                                                                   stop_words=stopwords.words("russian"),
                                                                   max_df=1500, min_df=15),
                                                   TfidfTransformer()).fit(data_reviews['text'])

Баланисировка данных

In [114]:
data_features = data_transformer.transform(data_reviews['text'])
from imblearn.over_sampling import RandomOverSampler
X, y = RandomOverSampler().fit_resample(data_features, data_reviews['label'])

In [115]:
print(f"positive label: {y[y==1].count()}")
print(f"negative label: {y[y==0].count()}")

positive label: 5710
negative label: 5710


Оценка по отложенной выборке

In [116]:
train_data, test_data, train_label, test_label = train_test_split(X, y, test_size=0.3, random_state=0)

In [117]:
model = LogisticRegression().fit(train_data, train_label)


In [118]:
print(f"Accuracy train: {metrics.accuracy_score(train_label, model.predict(train_data))}")
print(f"Accuracy test: {metrics.accuracy_score(test_label, model.predict(test_data))}")

Accuracy train: 0.9883662747060296
Accuracy test: 0.9708114419147694


Оценка по кросс-валиадации

In [119]:
cv = StratifiedKFold(n_splits=10)
cv_model = cross_val_score(LogisticRegression(), train_data, train_label, scoring='accuracy', cv = cv)

In [120]:
print(f'Mean accuracy:{cv_model.mean():.2f}')

Mean accuracy:0.97


Проверка на данных задания

In [121]:
#data_test = pd.read_csv('test.csv',engine='python')
with open('test.csv', 'rb') as f:
    data_test = f.read().decode('utf-8')

In [122]:
data_review = [' '.join([word.strip('\n <review> \r') for word in review.split(' ')]) for review in data_test.split('</review>')]

In [123]:
data_review

['Ужасно слабый аккумулятор, это основной минус этого аппарата, разряжается буквально за пару часов при включенном f и на макс подсветке, например если играть или смотреть видео, следовательно использовать можно только если есть постоянная возможность подзарядиться. Качества звука через динамик далеко не на высоте.Наблюдаются незначительные тормоза в некоторых приложениях и вообще в меню. Очень мало встроенной памяти, а приложения устанавливаются именно туда, с этим связанны неудобства - нужно постоянно переносить их на карту памяти.\nНесколько неудобно что нету отдельной кнопки для фото. Подумываю купить батарею большей емкость мб что нибудь измениться.',
 'ценанадежность-неубиваемостьдолго держит батарею 4 дня стабильно как телефон, 3-4 как плеер если постоянно долбиться в уши и звонить по паре часо на дню, игры и, конечно,  смс , в месяц около 200 шт набирается. Максимальное время работы 5 дней в щадящем режиме.2 simqwerty рулит -после нее набор смс на обычных сенсорниках и кнопочны

In [124]:
data_test = data_transformer.transform(data_review[:-1])
data_test

<100x15814 sparse matrix of type '<class 'numpy.float64'>'
	with 5849 stored elements in Compressed Sparse Row format>

In [125]:
predicted_test = model.predict(data_test)
len(predicted_test)

100

In [126]:
data_predicted = pd.DataFrame(predicted_test,columns=['y'])
data_predicted['y'].replace([1, 0], ['pos', 'neg'], inplace=True)
data_predicted.to_csv('data_predicted.csv', index_label='Id')